# Change model metadata
> Change model metadata without training new model

In [ ]:
#| default_exp postprocessing.model_metadata

In [ ]:
#| hide
%load_ext autoreload
%autoreload 2


In [ ]:
#| export
import sys
from pathlib import Path


In [ ]:
!which python

In [ ]:
#| export
#CV_TOOLS = Path(r'/home/ai_dsx.work/data/projects/cv_tools')
#sys.path.append(str(CV_TOOLS))


In [ ]:
#| export
from dotenv import load_dotenv
from fastcore.all import *


In [ ]:
#| export
from typing import Union, List, Optional, Dict, Any, Tuple
from pathlib import Path
from dataclasses import dataclass
import json
import torch
import numpy as np
import warnings
from datetime import datetime

# Anomalib imports
import anomalib
from anomalib.deploy import TorchInferencer, OpenVINOInferencer
from anomalib.models import (
    Padim, Patchcore, Cflow, Fastflow, Stfpm, 
    EfficientAd, Draem, ReverseDistillation,
    Dfkde, Dfm, Ganomaly, Cfa, Csflow, Dsr, Fre, Rkde, Uflow
)
from anomalib.utils.visualization.image import ImageVisualizer, VisualizationMode
from anomalib.data.utils import read_image

# Suppress warnings for cleaner output
warnings.filterwarnings('ignore')
import logging
logging.getLogger('anomalib').setLevel(logging.WARNING)

In [ ]:
# | export
from be_vision_ad_tools.inference.prediction_system import create_inference_poster_

In [ ]:
#| export
@dataclass
class ModelMetadata:
    """Configuration class for model metadata parameters."""
    image_threshold: Optional[float] = None
    pixel_threshold: Optional[float] = None
    pred_score_min: Optional[float] = None
    pred_score_max: Optional[float] = None
    anomaly_map_min: Optional[float] = None
    anomaly_map_max: Optional[float] = None
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert to dictionary, excluding None values."""
        return {k: v for k, v in self.__dict__.items() if v is not None}
    
    def update_from_dict(self, data: Dict[str, Any]) -> None:
        """Update metadata from dictionary."""
        for key, value in data.items():
            if hasattr(self, key) and value is not None:
                setattr(self, key, float(value))

In [ ]:
#| export
def _detect_model_class_from_filename(model_path: Path):
    """Auto-detect anomalib model class from filename."""
    filename_lower = model_path.name.lower()
    
    # Model mapping
    model_mapping = {
        'padim': Padim,
        'patchcore': Patchcore,
        'cflow': Cflow,
        'fastflow': Fastflow,
        'stfpm': Stfpm,
        'efficient': EfficientAd,
        'draem': Draem,
        'reverse': ReverseDistillation,
        'dfkde': Dfkde,
        'dfm': Dfm,
        'ganomaly': Ganomaly,
        'cfa': Cfa,
        'csflow': Csflow,
        'dsr': Dsr,
        'fre': Fre,
        'rkde': Rkde,
        'uflow': Uflow
    }
    
    for name, model_class in model_mapping.items():
        if name in filename_lower:
            print(f"🎯 Auto-detected model: {name.upper()}")
            return model_class
    
    # Default to PaDiM if can't detect
    print(f"⚠️  Could not auto-detect model type from filename, using PaDiM")
    return Padim

In [ ]:
tst_im_path = Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/images/bad')
print(f'tst im folder exists: {tst_im_path.exists()}')
model_path= Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/exports/TEST_MULITNODE_task_001_padim_resnet18_18_layer1-layer2-layer3/weights/torch/model.pt')
print(model_path.exists())
model_path_c= Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/checkpoints/TEST_MULITNODE_task_001_padim_resnet18_18_layer1-layer2-layer3/last.ckpt')
print(model_path_c.exists())

In [ ]:

_detect_model_class_from_filename(model_path)


In [ ]:

_detect_model_class_from_filename(model_path_c)

In [ ]:
#| export
def load_model_any_format(
    model_path: Union[str, Path], # Path to model file (.pt, .ckpt, .onnx)
    device: str = "auto" # Device to load model on ("auto", "cpu", "cuda")
) -> Tuple[Any, str, Dict[str, Any]]:
    """
    Load a model from any supported format and extract current metadata.
    
    """
    model_path = Path(model_path)
    
    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")
    
    # Auto-detect device
    if device == "auto":
        device = "cuda" if torch.cuda.is_available() else "cpu"
    
    model_ext = model_path.suffix.lower()
    current_metadata = {}
    
    print(f"📂 Loading {model_ext} model from: {model_path.name}")
    
    try:
        if model_ext in ['.pt', '.pth']:
            # Load PyTorch exported model
            model_data = torch.load(model_path, map_location=device, weights_only=False)
            
            # Extract metadata from model_data if available
            if isinstance(model_data, dict):
                # Check for metadata in the saved state
                if 'metadata' in model_data:
                    # Create a copy to avoid modifying the original
                    current_metadata = dict(model_data['metadata'])
                elif 'model_metadata' in model_data:
                    # Create a copy to avoid modifying the original
                    current_metadata = dict(model_data['model_metadata'])
                else:
                    # Try to extract from model state if it's a full model save
                    print("⚠️  No explicit metadata found in .pt file")
            
            return model_data, "torch", current_metadata
            
        elif model_ext == '.ckpt':
            # Load Lightning checkpoint
            model_class = _detect_model_class_from_filename(model_path)
            model = model_class.load_from_checkpoint(model_path, map_location=device)
            model.eval()
            
            # Extract current metadata from trained model
            if hasattr(model, 'image_threshold') and hasattr(model, 'normalization_metrics'):
                current_metadata = {
                    'image_threshold': float(model.image_threshold.value.item()) if hasattr(model.image_threshold.value, 'item') else float(model.image_threshold.value),
                    'pixel_threshold': float(model.pixel_threshold.value.item()) if hasattr(model.pixel_threshold.value, 'item') else float(model.pixel_threshold.value),
                    'pred_score_min': float(model.normalization_metrics.pred_scores.min.item()) if hasattr(model.normalization_metrics.pred_scores.min, 'item') else float(model.normalization_metrics.pred_scores.min),
                    'pred_score_max': float(model.normalization_metrics.pred_scores.max.item()) if hasattr(model.normalization_metrics.pred_scores.max, 'item') else float(model.normalization_metrics.pred_scores.max),
                    'anomaly_map_min': float(model.normalization_metrics.anomaly_maps.min.item()) if hasattr(model.normalization_metrics.anomaly_maps.min, 'item') else float(model.normalization_metrics.anomaly_maps.min),
                    'anomaly_map_max': float(model.normalization_metrics.anomaly_maps.max.item()) if hasattr(model.normalization_metrics.anomaly_maps.max, 'item') else float(model.normalization_metrics.anomaly_maps.max)
                }
            else:
                print("⚠️  Model doesn't have required metadata attributes")
            
            return model, "checkpoint", current_metadata
            
        elif model_ext in ['.onnx', '.xml']:
            # ONNX/OpenVINO models - metadata handling is limited
            print(f"📝 Loading {model_ext} model (limited metadata modification support)")
            
            # For ONNX/OpenVINO, we can't easily modify internal metadata
            # but we can create external metadata files
            metadata_file = model_path.with_suffix('.json')
            if metadata_file.exists():
                with open(metadata_file, 'r') as f:
                    current_metadata = json.load(f)
                print(f"📋 Loaded external metadata from: {metadata_file.name}")
            else:
                print("⚠️  No external metadata file found for ONNX/OpenVINO model")
            
            return str(model_path), model_ext[1:], current_metadata  # Return path as string
            
        else:
            raise ValueError(f"Unsupported model format: {model_ext}. Supported: .pt, .ckpt, .onnx, .xml")
            
    except Exception as e:
        raise RuntimeError(f"Failed to load model {model_path}: {str(e)}")

In [ ]:
print(model_path)
print(Path(model_path).exists())
print(Path(model_path_c).exists())
model_obj, model_f, ori_metadata = loaded_model = load_model_any_format(model_path)
model_obj_c, model_f_c, ori_metadata_c = loaded_model = load_model_any_format(model_path_c)

loaded_model_c = load_model_any_format(model_path_c)

### Step 2: Modify Metadata

In [ ]:
#| export
def _validate_metadata_ranges(metadata_dict: Dict[str, Any]) -> None:
    """Validate metadata value ranges."""
    for key, value in metadata_dict.items():
        if not isinstance(value, (int, float)):
            raise ValueError(f"Metadata value for {key} must be numeric, got {type(value)}")
            
        # Basic range validation
        if key.endswith('_threshold') and (value < 0 or value > 100):
            print(f"⚠️  Warning: {key} = {value} is outside typical range [0, 100]")
            
        if key.endswith('_min') and key.endswith('_max'):
            # This would need more sophisticated validation for min/max pairs
            pass

In [ ]:
#| export
def modify_model_metadata(
    model_object: Any, # Loaded model object
    model_format: str, # Model format ("torch", "checkpoint", "onnx", "xml")
    new_metadata: Union[ModelMetadata, Dict[str, Any]], # New metadata values
    validate_ranges: bool = True # Whether to validate metadata value ranges
) -> Tuple[Any, Dict[str, Any]]:
    """
    Modify model metadata in-place with validation.
    
    """
    # Convert to dict if ModelMetadata object
    if isinstance(new_metadata, ModelMetadata):
        metadata_dict = new_metadata.to_dict()
    else:
        metadata_dict = dict(new_metadata)
    
    # Validate metadata ranges
    if validate_ranges:
        _validate_metadata_ranges(metadata_dict)
    
    applied_metadata = {}
    
    print(f"🔧 Modifying metadata for {model_format} model...")
    
    try:
        if model_format == "checkpoint":
            # Modify Lightning checkpoint model
            for key, value in metadata_dict.items():
                if key == 'image_threshold' and hasattr(model_object, 'image_threshold'):
                    model_object.image_threshold.value = torch.tensor(value, device=model_object.device)
                    applied_metadata[key] = value
                    print(f"  ✅ Updated image_threshold: {value}")
                    
                elif key == 'pixel_threshold' and hasattr(model_object, 'pixel_threshold'):
                    model_object.pixel_threshold.value = torch.tensor(value, device=model_object.device)
                    applied_metadata[key] = value
                    print(f"  ✅ Updated pixel_threshold: {value}")
                    
                elif key.startswith('pred_score_') and hasattr(model_object, 'normalization_metrics'):
                    if key == 'pred_score_min':
                        model_object.normalization_metrics.pred_scores.min = torch.tensor(value, device=model_object.device)
                        applied_metadata[key] = value
                        print(f"  ✅ Updated pred_score_min: {value}")
                    elif key == 'pred_score_max':
                        model_object.normalization_metrics.pred_scores.max = torch.tensor(value, device=model_object.device)
                        applied_metadata[key] = value
                        print(f"  ✅ Updated pred_score_max: {value}")
                        
                elif key.startswith('anomaly_map_') and hasattr(model_object, 'normalization_metrics'):
                    if key == 'anomaly_map_min':
                        model_object.normalization_metrics.anomaly_maps.min = torch.tensor(value, device=model_object.device)
                        applied_metadata[key] = value
                        print(f"  ✅ Updated anomaly_map_min: {value}")
                    elif key == 'anomaly_map_max':
                        model_object.normalization_metrics.anomaly_maps.max = torch.tensor(value, device=model_object.device)
                        applied_metadata[key] = value
                        print(f"  ✅ Updated anomaly_map_max: {value}")
                else:
                    print(f"  ⚠️  Skipped {key}: Not supported or attribute not found")
                    
        elif model_format == "torch":
            # Modify PyTorch exported model data
            if isinstance(model_object, dict):
                # Ensure metadata section exists
                if 'metadata' not in model_object:
                    model_object['metadata'] = {}
                
                # Update metadata
                model_object['metadata'].update(metadata_dict)
                applied_metadata = dict(metadata_dict)
                
                for key, value in metadata_dict.items():
                    print(f"  ✅ Updated {key}: {value}")
            else:
                print("  ⚠️  PyTorch model is not in expected dictionary format")
                
        elif model_format in ["onnx", "xml"]:
            # For ONNX/OpenVINO, we'll handle metadata externally
            applied_metadata = dict(metadata_dict)
            print(f"  📝 Metadata will be saved externally for {model_format} model")
            for key, value in metadata_dict.items():
                print(f"  ✅ Prepared {key}: {value}")
        else:
            raise ValueError(f"Unsupported model format: {model_format}")
            
        return model_object, applied_metadata
        
    except Exception as e:
        raise RuntimeError(f"Failed to modify model metadata: {str(e)}")


In [ ]:
new_metadata = ModelMetadata(
    image_threshold=0.7,      # Increase detection threshold
    pred_score_min=0.0,       # Set normalization range
    pred_score_max=20.0,
    anomaly_map_min=0.0,      # Set anomaly map normalization range
    anomaly_map_max=10.0,
    pixel_threshold=0.7      # Set pixel-level threshold

)

In [ ]:
modified_model, applied_metdata = modify_model_metadata(
    model_object=model_obj,
    model_format='torch',#'checkpoint',
    new_metadata=new_metadata,
    validate_ranges=True
)




In [ ]:
#modified_model_p = Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/modified_model')
#Path(modified_model_p).mkdir(parents=True, exist_ok=True)
#torch.save(modified_model, f'{modified_model_p}/model.pt')

### Step 3: Save modified model and metadata

In [ ]:
#| export
def save_model_with_metadata(
    model_object: Any, # Modified model object
    model_format: str, # Model format ("torch", "checkpoint", "onnx", "xml")
    output_path: Union[str, Path], # Path to save the modified model
    metadata: Dict[str, Any], # Metadata that was applied
    create_backup: bool = True, # Whether to create backup of original
    backup_suffix: str = "_backup" # Suffix for backup files
) -> Dict[str, Any]:
    """
    Save model with modified metadata.
    """
    output_path = Path(output_path)
    output_path.parent.mkdir(parents=True, exist_ok=True)
    
    results = {
        'saved_model_path': str(output_path),
        'backup_path': None,
        'metadata_file': None,
        'format': model_format,
        'metadata': metadata,
        'timestamp': datetime.now().isoformat()
    }
    
    print(f"💾 Saving {model_format} model to: {output_path.name}")
    
    try:
        # Create backup if requested and file exists
        if create_backup and output_path.exists():
            backup_path = output_path.with_stem(f"{output_path.stem}{backup_suffix}")
            backup_path.write_bytes(output_path.read_bytes())
            results['backup_path'] = str(backup_path)
            print(f"📁 Created backup: {backup_path.name}")
        
        if model_format == "checkpoint":
            # Save Lightning checkpoint
            torch.save(model_object.state_dict(), output_path)
            print(f"  ✅ Saved checkpoint model")
            
        elif model_format == "torch":
            # Save PyTorch model
            torch.save(model_object, output_path)
            print(f"  ✅ Saved PyTorch model")
            
        elif model_format in ["onnx", "xml"]:
            # For ONNX/OpenVINO, save external metadata file
            metadata_file = output_path.with_suffix('.json')
            with open(metadata_file, 'w') as f:
                json.dump({
                    'model_metadata': metadata,
                    'modified_timestamp': datetime.now().isoformat(),
                    'original_model': str(output_path)
                }, f, indent=2)
            
            results['metadata_file'] = str(metadata_file)
            print(f"  ✅ Saved external metadata: {metadata_file.name}")
            
            # Note: Original ONNX/OpenVINO model file should be copied if different location
            if isinstance(model_object, str) and Path(model_object) != output_path:
                # Copy original model file
                output_path.write_bytes(Path(model_object).read_bytes())
                print(f"  ✅ Copied model file")
        
        print(f"🎉 Successfully saved model with modified metadata!")
        return results
        
    except Exception as e:
        raise RuntimeError(f"Failed to save model: {str(e)}")

In [ ]:
save_res = save_model_with_metadata(
    model_object=modified_model,
    model_format='torch',
    output_path=Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/modified_model/model_m.pt'),
    metadata=applied_metdata,
    create_backup=True
)

In [ ]:
save_res

### Step 4: Visualize results with modified metadata

In [ ]:
#| export
def create_metadata_comparison_poster(
    original_model_path: Union[str, Path], # Path to original model
    modified_model_path: Union[str, Path], # Path to modified model  
    test_images: List[Union[str, Path]], # Test images for comparison
    output_dir: Union[str, Path], # Output directory
    metadata_changes: Dict[str, Any], # Summary of metadata changes
    poster_title_prefix: str = "Metadata Comparison", # Title prefix
    poster_rows:int=3, # Number of poster rows
    poster_cols:int=3, # Number of poster columns
    image_size_in_poster: Tuple[int, int]=(256, 256), # Size of images in poster
    include_heatmap: bool = True, # Whether to include heatmap poster
    include_image_poster: bool = False, # Whether to include image poster
    include_anomaly_poster: bool = False, # Whether to include anomaly poster
    show_plot: bool = False # Whether to display the plot
) -> Dict[str, Any]:
    """
    Create model metadata comparison posters 
    
        
    Example:
        ```python
        results = create_metadata_comparison_poster(
            original_model_path="models/original.pt",
            modified_model_path="models/modified.pt", 
            test_images=["test1.jpg", "test2.jpg"],
            output_dir="comparison_results",
            metadata_changes={
                "image_threshold": {"old": 0.5, "new": 0.7},
                "pred_score_max": {"old": 1.0, "new": 20.0}
            }
        )
        ```
    """
    
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    
    print(f"   Original model: {Path(original_model_path).name}")
    print(f"   Modified model: {Path(modified_model_path).name}")
    print(f"   Test images: {len(test_images)}")
    
    # Create title with metadata changes summary
    changes_summary = ", ".join([
        f"{k}: {v['old']}→{v['new']}" for k, v in metadata_changes.items()
    ])
    
    try:
        # Create poster for original model
        print("\\n📊 Creating original model poster...")
        original_poster_results = create_inference_poster_(
            model_path=original_model_path,
            test_images=test_images,
            output_folder=output_dir / "original_model",
            poster_title=f"{poster_title_prefix} - Original Model",
            poster_rows=poster_rows,
            poster_cols=poster_cols,
            include_heatmap_poster=include_heatmap,
            include_image_poster=include_image_poster,
            include_anomaly_poster=include_anomaly_poster,
            image_size_in_poster=image_size_in_poster,
            device="auto"
        )
        
        # Create poster for modified model  
        print("\\n📊 Creating modified model poster...")
        modified_poster_results = create_inference_poster_(
            model_path=modified_model_path,
            test_images=test_images,
            output_folder=output_dir / "modified_model",
            poster_title=f"{poster_title_prefix} - Modified Model ({changes_summary})",
            poster_rows=poster_rows,
            poster_cols=poster_cols,
            include_heatmap_poster=include_heatmap,
            include_image_poster=include_image_poster,
            include_anomaly_poster=include_anomaly_poster,
            image_size_in_poster=image_size_in_poster,
            device="auto",
            show_plot=True
        )
        
        # Create summary comparison document
        summary_path = output_dir / "comparison_summary.json"
        summary_data = {
            "metadata_changes": metadata_changes,
            "original_model": {
                "path": str(original_model_path),
                "posters": original_poster_results.get("posters", []),
                "statistics": original_poster_results.get("statistics", {})
            },
            "modified_model": {
                "path": str(modified_model_path), 
                "posters": modified_poster_results.get("posters", []),
                "statistics": modified_poster_results.get("statistics", {})
            },
            "test_images": [str(img) for img in test_images],
            "timestamp": datetime.now().isoformat()
        }
        
        import json
        with open(summary_path, 'w') as f:
            json.dump(summary_data, f, indent=2)
        
        print(f"\\n🎉 Metadata comparison completed!")
        print(f"   Original posters: {len(original_poster_results.get('posters', []))}")
        print(f"   Modified posters: {len(modified_poster_results.get('posters', []))}")
        print(f"   Summary: {summary_path}")
        print(f"   Output directory: {output_dir}")
        
        return {
            "success": True,
            "original_results": original_poster_results,
            "modified_results": modified_poster_results,
            "summary_file": str(summary_path),
            "output_directory": str(output_dir),
            "metadata_changes": metadata_changes
        }
        
    except Exception as e:
        error_msg = f"Failed to create metadata comparison: {str(e)}"
        print(f"❌ {error_msg}")
        return {
            "success": False,
            "error": error_msg
        }

In [ ]:
# Check what keys are available in ori_metadata
print("Available keys in ori_metadata:")
for key in ori_metadata.keys():
    print(f"  '{key}': {ori_metadata[key]}")

# Access the values with correct key names
anomaly_map_min_original = ori_metadata.get('anomaly_maps.min', None)  # Note: 'anomaly_maps.min' not 'anomaly_map.min'
anomaly_map_max_original = ori_metadata.get('anomaly_maps.max', None)
pred_score_min_original = ori_metadata.get('pred_scores.min', None)
pred_score_max_original = ori_metadata.get('pred_scores.max', None)

# Create the metadata_changes dictionary with correct original values
metadata_changes = {
    "image_threshold": {"old": ori_metadata.get('image_threshold', None), "new": new_metadata.image_threshold},
    "pred_score_min": {"old": pred_score_min_original, "new": new_metadata.pred_score_min},
    "pred_score_max": {"old": pred_score_max_original, "new": new_metadata.pred_score_max},
    "anomaly_map_min": {"old": anomaly_map_min_original, "new": new_metadata.anomaly_map_min},
    "anomaly_map_max": {"old": anomaly_map_max_original, "new": new_metadata.anomaly_map_max},
    "pixel_threshold": {"old": ori_metadata.get('pixel_threshold', None), "new": new_metadata.pixel_threshold}
}

print(f"\nMetadata changes:")
for key, change in metadata_changes.items():
    print(f"  {key}: {change['old']} → {change['new']}")

In [ ]:
ori_metadata.get('anomaly_map.min', None)

In [ ]:
new_metadata

In [ ]:
create_metadata_comparison_poster(
    original_model_path=Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/modified_model/model.pt'),
    modified_model_path=Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/modified_model/model_m.pt'),
    test_images=[Path(tst_im_path, i) for i in os.listdir(tst_im_path)],

    output_dir=Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/modified_model/comparison_posters'),
    metadata_changes={
        "image_threshold": {"old": ori_metadata.get('image_threshold', None), "new": new_metadata.image_threshold},
        "pred_score_min": {"old": ori_metadata.get('pred_score_min', None), "new": new_metadata.pred_score_min},
        "pred_score_max": {"old": ori_metadata.get('pred_score_max', None), "new": new_metadata.pred_score_max},
        "anomaly_map_min": {"old": ori_metadata.get('anomaly_maps.min', None), "new": new_metadata.anomaly_map_min},
        "anomaly_map_max": {"old": ori_metadata.get('anomaly_maps.max', None), "new": new_metadata.anomaly_map_max},
        "pixel_threshold": {"old": ori_metadata.get('pixel_threshold', None), "new": new_metadata.pixel_threshold}
    },
    poster_rows=1,
    poster_cols=2,
    show_plot=True
)

# Step 5: all Together

In [ ]:
#| export
def modify_model_metadata_complete(
    model_path: Union[str, Path], # Path to the input model
    new_metadata: Union[ModelMetadata, Dict[str, Any]], # New metadata to apply
    output_path: Optional[Union[str, Path]] = None, # Output path (default: same as input with suffix)
    create_backup: bool = True, # Whether to create backup
    create_poster: bool = False, # Whether to create comparison poster
    test_images: Optional[Union[str, List[Union[str, Path]]]] = None, # Test images for poster
    device: str = "auto", # Device for model operations
    include_heatmap: bool = True, # Whether to include heatmap in poster
    include_image_poster: bool = False, # Whether to include image poster
    include_anomaly_poster: bool = False, # Whether to include anomaly poster
    image_size_in_poster: Tuple[int, int] = (256, 256), # Size of images in poster
    poster_rows: int = 3, # Number of poster rows
    poster_cols: int = 3, # Number of poster columns
    poster_title_prefix: str = "Metadata Comparison" # Title prefix for the poster
) -> Dict[str, Any]:
    """
    Complete workflow to modify model metadata with optional poster creation.
    
    This is the main function that combines all operations:
    1. Load model from any format (.pt, .ckpt, .onnx)
    2. Extract current metadata
    3. Apply new metadata values
    4. Save modified model
    5. Optionally create comparison poster
    
    Example:
        ```python
        # Modify threshold values
        new_meta = ModelMetadata(
            image_threshold=0.7,
            pixel_threshold=0.5,
            pred_score_min=0.0,
            pred_score_max=1.0
        )
        
        results = modify_model_metadata_complete(
            model_path="model.pt",
            new_metadata=new_meta,
            create_poster=True,
            test_images=["test1.jpg", "test2.jpg"] or "test_image_folder"
        )
        ```
    """
    model_path = Path(model_path)
    
    # Validate inputs
    if not model_path.exists():
        raise FileNotFoundError(f"Model file not found: {model_path}")
    
    if create_poster and not test_images:
        raise ValueError("test_images required when create_poster=True")
    
    # Generate output path if not provided
    if output_path is None:
        output_path = model_path.with_stem(f"{model_path.stem}_modified")
    else:
        output_path = Path(output_path)
    
    print(f"🚀 Starting complete metadata modification workflow...")
    print(f"📂 Input model: {model_path.name}")
    print(f"💾 Output model: {output_path.name}")
    
    results = {
        'input_model': str(model_path),
        'output_model': str(output_path),
        'success': False,
        'operations': [],
        'timestamp': datetime.now().isoformat()
    }
    
    try:
        # Step 1: Load model and extract current metadata
        print("\n📖 Step 1: Loading model...")
        model_object, model_format, original_metadata = load_model_any_format(model_path, device)
        results['original_metadata'] = original_metadata
        results['model_format'] = model_format
        results['operations'].append("model_loaded")
        
        # Step 2: Modify metadata
        print("\\n🔧 Step 2: Modifying metadata...")
        modified_model, applied_metadata = modify_model_metadata(
            model_object, model_format, new_metadata, validate_ranges=True
        )
        results['applied_metadata'] = applied_metadata
        results['operations'].append("metadata_modified")
        
        # Step 3: Save modified model
        print("\\n💾 Step 3: Saving modified model...")
        save_results = save_model_with_metadata(
            modified_model, model_format, output_path, applied_metadata, create_backup
        )
        results.update(save_results)
        results['operations'].append("model_saved")
        
        print('Before step 4 check')
        print(f'create_poster: {create_poster}, test_images: {test_images}')
        # Step 4: Create poster if requested
        if create_poster and test_images:
            if isinstance(test_images, list):
                test_images = [str(img) for img in test_images]
            if isinstance(test_images, L):
                test_images = [Path(img).as_posix() for img in test_images]
            if isinstance(test_images, str):
                test_images = [Path(i).as_posix() for i in Path(test_images).ls()]
            print("\\n🎨 Step 4: Creating comparison poster...")
            poster_dir = output_path.parent / "posters"
            print(f"{'#' * 10} poster dir: {poster_dir}")
            print(applied_metadata)
            print(f"{'#' * 10} poster dir: {poster_dir}")
            
            # Build metadata changes dictionary using helper function
            # This handles partial updates properly by falling back to original values
            metadata_changes = build_metadata_changes_dict(
                applied_metadata=applied_metadata,
                original_metadata=original_metadata,
                include_unchanged=True  # Include all values for complete comparison
            )
            
            # Validate the metadata changes dictionary
            validation_results = validate_metadata_changes_dict(
                metadata_changes=metadata_changes,
                allow_none_values=True  # Allow None values for missing metadata
            )
            
            if not validation_results['valid']:
                print(f"⚠️  Metadata changes validation issues: {validation_results['issues']}")
            else:
                print(f"✅ Metadata changes built successfully:")
                print(f"   - Total parameters: {validation_results['total_changes']}")
                print(f"   - Modified: {validation_results['modified_count']}")
                print(f"   - Unchanged: {validation_results['unchanged_count']}")
            
            poster_results = create_metadata_comparison_poster(
                original_model_path=model_path,
                modified_model_path=save_results['saved_model_path'],
                test_images=test_images,
                output_dir=poster_dir,
                metadata_changes=metadata_changes,
                poster_rows=poster_rows,
                poster_cols=poster_cols,
                show_plot=False,
                image_size_in_poster=image_size_in_poster,
                include_heatmap=include_heatmap,
                include_image_poster=include_image_poster,
                include_anomaly_poster=include_anomaly_poster,
                poster_title_prefix="Metadata Comparison"
            )
            results['poster_results'] = poster_results
            results['operations'].append("poster_created")
        
        results['success'] = True
        print(f"\\n🎉 Complete workflow finished successfully!")
        print(f"✅ Operations completed: {', '.join(results['operations'])}")
        
        return results
        
    except Exception as e:
        results['error'] = str(e)
        results['success'] = False
        print(f"\\n❌ Workflow failed: {str(e)}")
        raise RuntimeError(f"Complete metadata modification failed: {str(e)}")

In [ ]:
#| export  
def build_metadata_changes_dict(
    applied_metadata: Union[ModelMetadata, Dict[str, Any]], # Applied/modified metadata
    original_metadata: Dict[str, Any], # Original metadata from model
    include_unchanged: bool = True # Whether to include unchanged values
) -> Dict[str, Dict[str, Any]]:
    """
    Build a metadata changes dictionary that properly handles partial updates.
    
    """
    
    def get_metadata_value(
        applied_meta, 
        original_meta, 
        key_mapping):
        """
        Get metadata value from applied_metadata with fallback to original_metadata.
        
        """
        applied_key = key_mapping['applied_key']
        original_key = key_mapping['original_key']
        
        # Try to get from applied metadata (handle both dict and object)
        if hasattr(applied_meta, applied_key):
            # Object-style access (for ModelMetadata objects)
            new_value = getattr(applied_meta, applied_key, None)
        elif isinstance(applied_meta, dict) and applied_key in applied_meta:
            # Dict-style access
            new_value = applied_meta[applied_key]
        else:
            # Fallback to original metadata value
            new_value = original_meta.get(original_key, None)
        
        return new_value
    
    # Define mappings between applied metadata keys and original metadata keys
    metadata_mappings = {
        "image_threshold": {
            'applied_key': 'image_threshold',
            'original_key': 'image_threshold'
        },
        "pred_score_min": {
            'applied_key': 'pred_score_min', 
            'original_key': 'pred_scores.min'
        },
        "pred_score_max": {
            'applied_key': 'pred_score_max',
            'original_key': 'pred_scores.max'
        },
        "anomaly_map_min": {
            'applied_key': 'anomaly_map_min',
            'original_key': 'anomaly_maps.min'
        },
        "anomaly_map_max": {
            'applied_key': 'anomaly_map_max',
            'original_key': 'anomaly_maps.max'
        },
        "pixel_threshold": {
            'applied_key': 'pixel_threshold',
            'original_key': 'pixel_threshold'
        }
    }
    
    metadata_changes = {}
    
    for change_key, mapping in metadata_mappings.items():
        old_value = original_metadata.get(mapping['original_key'], None)
        new_value = get_metadata_value(applied_metadata, original_metadata, mapping)
        
        # Include in result if changed or if include_unchanged is True
        if include_unchanged or old_value != new_value:
            metadata_changes[change_key] = {
                "old": old_value,
                "new": new_value
            }
    
    return metadata_changes



In [ ]:
# TODO:
# 1. test build_metadata_changes_dict with include_unchanged=False
# 2. test validate_metadata_changes_dict with allow_none_values=True

In [ ]:
#| export
def validate_metadata_changes_dict(
    metadata_changes: Dict[str, Dict[str, Any]], # Metadata changes dictionary
    allow_none_values: bool = False # Whether None values are acceptable
) -> Dict[str, Any]:
    """
    Validate a metadata changes dictionary and return validation results.
    
    """
    validation_results = {
        'valid': True,
        'total_changes': len(metadata_changes),
        'modified_count': 0,
        'unchanged_count': 0,
        'none_value_count': 0,
        'issues': []
    }
    
    for param_name, change_data in metadata_changes.items():
        # Check structure
        if not isinstance(change_data, dict) or 'old' not in change_data or 'new' not in change_data:
            validation_results['valid'] = False
            validation_results['issues'].append(f"Invalid structure for {param_name}")
            continue
        
        old_val = change_data['old']
        new_val = change_data['new']
        
        # Count None values
        if old_val is None or new_val is None:
            validation_results['none_value_count'] += 1
            if not allow_none_values:
                validation_results['valid'] = False
                validation_results['issues'].append(f"None value found in {param_name}")
        
        # Count changes
        if old_val != new_val:
            validation_results['modified_count'] += 1
        else:
            validation_results['unchanged_count'] += 1
    
    return validation_results

In [ ]:
#| export
@call_parse
def modify_model_metadata_cli(
    model_path: Union[str, Path]=None, # Path to the input model
    output_path: Optional[Union[str, Path]] = None, # Output path (default: same as input with suffix)
    config_file: str = None, # Optional config file (JSON/YAML) with metadata parameters
    # Individual metadata parameters (CLI-friendly) - ignored if config_file is provided
    image_threshold: Optional[float] = None, # Image-level anomaly threshold
    pixel_threshold: Optional[float] = None, # Pixel-level anomaly threshold
    pred_score_min: Optional[float] = None, # Prediction score minimum for normalization
    pred_score_max: Optional[float] = None, # Prediction score maximum for normalization
    anomaly_map_min: Optional[float] = None, # Anomaly map minimum for normalization
    anomaly_map_max: Optional[float] = None, # Anomaly map maximum for normalization
    # General options
    create_backup: bool = False, # Whether to create backup
    create_poster: bool = True, # Whether to create comparison poster
    test_images_folder: Optional[Union[str, Path]] = None, # Folder containing test images
    image_extensions: List[str] = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff'], # Valid image extensions
    max_test_images: int = 20, # Maximum number of test images to use
    device: str = "auto", # Device for model operations
    include_heatmap: bool = True, # Whether to include heatmap in poster
    include_image_poster: bool = False, # Whether to include image poster
    include_anomaly_poster: bool = False, # Whether to include anomaly poster
    image_size_in_poster_width: int = 256, # Width of images in poster
    image_size_in_poster_height: int = 256, # Height of images in poster
    poster_rows: int = 3, # Number of poster rows
    poster_cols: int = 3, # Number of poster columns
    poster_title_prefix: str = "Metadata Comparison" # Title prefix for the poster
) -> Dict[str, Any]:
    """
    Command-line interface wrapper for model metadata modification.
    Example:
        ```python
        # Method 1: CLI-style usage with individual parameters
        results = modify_model_metadata_cli(
            model_path="model.pt",
            image_threshold=0.7,
            pixel_threshold=0.5,
            pred_score_max=2.0,
            create_poster=True,
            test_images_folder="test_images"
        )
        
        # Method 2: Using config file
        results = modify_model_metadata_cli(
            model_path="model.pt",
            config_file="metadata_config.json",
            create_poster=True,
            test_images_folder="test_images"
        )
        ```
    """
    
    # Handle config file loading if provided
    if config_file:
        config_file = Path(config_file)
        if not config_file.exists():
            raise FileNotFoundError(f"Config file not found: {config_file}")
        
        print(f"📄 Loading configuration from: {config_file.name}")
        
        try:
            # Load config based on file extension
            if config_file.suffix.lower() == '.json':
                import json
                with open(config_file, 'r') as f:
                    config_data = json.load(f)
            elif config_file.suffix.lower() in ['.yaml', '.yml']:
                try:
                    import yaml
                    with open(config_file, 'r') as f:
                        config_data = yaml.safe_load(f)
                except ImportError:
                    raise ImportError("PyYAML is required to load YAML config files. Install with: pip install pyyaml")
            else:
                raise ValueError(f"Unsupported config file format: {config_file.suffix}. Use .json or .yaml/.yml")


            #Creating first default and then overwriting with cli values
            image_extensions = ['.jpg', '.jpeg', '.png', '.bmp', '.tiff']


             # Extract model_path from config (REQUIRED)
            if 'model_path' not in config_data or config_data['model_path'] is None:
                raise ValueError("model_path must be specified in the config file")
            model_path = config_data['model_path']
            
            # Extract output_path from config (optional)
            output_path = config_data.get('output_path', None)
            
            # Extract metadata from config
            metadata_section = config_data.get('metadata', config_data)  # Allow 'metadata' section or root level
            
            # Build metadata dictionary from config
            new_metadata = {}
            metadata_keys = ['image_threshold', 'pixel_threshold', 'pred_score_min', 
                           'pred_score_max', 'anomaly_map_min', 'anomaly_map_max']
            
            for key in metadata_keys:
                if key in metadata_section and metadata_section[key] is not None:
                    new_metadata[key] = float(metadata_section[key])
            
            # Override other settings from config if present
            if 'create_backup' in config_data:
                create_backup = bool(config_data['create_backup'])
            if 'create_poster' in config_data:
                create_poster = bool(config_data['create_poster'])
            if 'test_images_folder' in config_data:
                test_images_folder = config_data['test_images_folder']
            if 'max_test_images' in config_data:
                max_test_images = int(config_data['max_test_images'])
            if 'device' in config_data:
                device = str(config_data['device'])

            if 'image_extensions' in config_data:
                image_extensions = list(config_data['image_extensions'])
            
            # Poster settings from config
            if 'poster_settings' in config_data:
                poster_cfg = config_data['poster_settings']
                if 'include_heatmap' in poster_cfg:
                    include_heatmap = bool(poster_cfg['include_heatmap'])
                if 'include_image_poster' in poster_cfg:
                    include_image_poster = bool(poster_cfg['include_image_poster'])
                if 'include_anomaly_poster' in poster_cfg:
                    include_anomaly_poster = bool(poster_cfg['include_anomaly_poster'])
                if 'image_size_width' in poster_cfg:
                    image_size_in_poster_width = int(poster_cfg['image_size_width'])
                if 'image_size_height' in poster_cfg:
                    image_size_in_poster_height = int(poster_cfg['image_size_height'])
                if 'rows' in poster_cfg:
                    poster_rows = int(poster_cfg['rows'])
                if 'cols' in poster_cfg:
                    poster_cols = int(poster_cfg['cols'])
                if 'title_prefix' in poster_cfg:
                    poster_title_prefix = str(poster_cfg['title_prefix'])
            
            print(f"✅ Config loaded: {len(new_metadata)} metadata parameters")
            
        except Exception as e:
            raise RuntimeError(f"Failed to load config file {config_file}: {str(e)}")
    
    else:
        # Build metadata dictionary from individual parameters
        metadata_params = {
            'image_threshold': image_threshold,
            'pixel_threshold': pixel_threshold,
            'pred_score_min': pred_score_min,
            'pred_score_max': pred_score_max,
            'anomaly_map_min': anomaly_map_min,
            'anomaly_map_max': anomaly_map_max
        }
        
        # Filter out None values to create final metadata
        new_metadata = {k: v for k, v in metadata_params.items() if v is not None}
        print(f"🔧 CLI metadata parameters: {new_metadata}")
    
    # Validate that at least one metadata parameter was provided
    if not new_metadata:
        raise ValueError(
            "At least one metadata parameter must be specified either as individual parameters or in config file: "
            "image_threshold, pixel_threshold, pred_score_min, pred_score_max, "
            "anomaly_map_min, or anomaly_map_max"
        )
    
    # Validate inputs
    if create_poster and not test_images_folder:
        raise ValueError("test_images_folder is required when create_poster=True")
    
    test_images = None
    
    # Process test images folder if provided
    if test_images_folder:
        test_images_folder = Path(test_images_folder)
        
        if not test_images_folder.exists():
            raise FileNotFoundError(f"Test images folder not found: {test_images_folder}")
        
        if not test_images_folder.is_dir():
            raise ValueError(f"test_images_folder must be a directory: {test_images_folder}")
        
        print(f"🔍 Scanning test images folder: {test_images_folder}")
        
        # Find all image files in the folder
        test_images = []
        for ext in image_extensions:
            # Case-insensitive search
            pattern_lower = f"*{ext.lower()}"
            pattern_upper = f"*{ext.upper()}"
            
            test_images.extend(test_images_folder.glob(pattern_lower))
            test_images.extend(test_images_folder.glob(pattern_upper))
        
        # Remove duplicates and sort
        test_images = sorted(list(set(test_images)))
        
        # Limit number of images
        if len(test_images) > max_test_images:
            print(f"⚠️  Found {len(test_images)} images, limiting to {max_test_images}")
            test_images = test_images[:max_test_images]
        
        if not test_images:
            print(f"⚠️  No valid images found in {test_images_folder}")
            print(f"   Supported extensions: {image_extensions}")
            if create_poster:
                raise ValueError(f"No test images found in {test_images_folder}")
        else:
            print(f"✅ Found {len(test_images)} test images")
    
    # Convert separate width/height to tuple
    image_size_in_poster = (image_size_in_poster_width, image_size_in_poster_height)
    
    # Call the main function with processed parameters
    print(f"🚀 Calling modify_model_metadata_complete with processed parameters...")
    
    return modify_model_metadata_complete(
        model_path=model_path,
        new_metadata=new_metadata,
        output_path=output_path,
        create_backup=create_backup,
        create_poster=create_poster,
        test_images=test_images,
        device=device,
        include_heatmap=include_heatmap,
        include_image_poster=include_image_poster,
        include_anomaly_poster=include_anomaly_poster,
        image_size_in_poster=image_size_in_poster,
        poster_rows=poster_rows,
        poster_cols=poster_cols,
        poster_title_prefix=poster_title_prefix
    )

In [ ]:
model_path = Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/models/exports/TEST_MULITNODE_task_000_padim_resnet18_18_layer1/weights/torch/model.pt')
test_images = Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/images/bad')
modify_model_metadata_cli(
    model_path=model_path,
    image_threshold=10,
    pixel_threshold=10,
    pred_score_max=10.0,
    anomaly_map_min=0,
    anomaly_map_max=12.0,
    create_poster=True,
    test_images_folder=test_images,
    poster_rows=1,
    poster_cols=2
)

In [ ]:
yaml_path= Path(r'/home/ai_dsx.work/data/projects/AD_tool_test/model_metadata_change.yaml')
yaml_path.exists()

In [ ]:
import yaml
with open(yaml_path, 'r') as f:
    yaml_data = yaml.safe_load(f)

In [ ]:
res= modify_model_metadata_cli(
    config_file=yaml_path,
)

In [ ]:
#| export
def create_metadata_config_template(
    output_path: Union[str, Path], # Path to save the config template
    format_type: str = "json", # Config format ("json" or "yaml")
    include_example_values: bool = True # Whether to include example values
) -> None:
    """
    Create a template configuration file for metadata modification.
    
    Args:
        output_path: Path where to save the config template
        format_type: Format of config file ("json" or "yaml")  
        include_example_values: Whether to include example values or null/None
        
    Example:
        ```python
        # Create JSON template
        create_metadata_config_template(
            "metadata_config.json", 
            format_type="json",
            include_example_values=True
        )
        
        # Create YAML template
        create_metadata_config_template(
            "metadata_config.yaml",
            format_type="yaml", 
            include_example_values=False
        )
        ```
    """
    
    output_path = Path(output_path)
    
    # Template configuration data
    if include_example_values:
        config_data = {
            "metadata": {
                "image_threshold": 0.7,
                "pixel_threshold": 0.5,
                "pred_score_min": 0.0,
                "pred_score_max": 2.0,
                "anomaly_map_min": 0.0,
                "anomaly_map_max": 10.0
            },
            "create_backup": True,
            "create_poster": True,
            "test_images_folder": "test_images",
            "max_test_images": 20,
            "device": "auto",
            "poster_settings": {
                "include_heatmap": True,
                "include_image_poster": False,
                "include_anomaly_poster": False,
                "image_size_width": 256,
                "image_size_height": 256,
                "rows": 3,
                "cols": 3,
                "title_prefix": "Metadata Comparison"
            }
        }
    else:
        config_data = {
            "metadata": {
                "image_threshold": None,
                "pixel_threshold": None,
                "pred_score_min": None,
                "pred_score_max": None,
                "anomaly_map_min": None,
                "anomaly_map_max": None
            },
            "create_backup": True,
            "create_poster": False,
            "test_images_folder": None,
            "max_test_images": 20,
            "device": "auto",
            "poster_settings": {
                "include_heatmap": True,
                "include_image_poster": False,
                "include_anomaly_poster": False,
                "image_size_width": 256,
                "image_size_height": 256,
                "rows": 3,
                "cols": 3,
                "title_prefix": "Metadata Comparison"
            }
        }
    
    # Save config file
    try:
        if format_type.lower() == "json":
            import json
            with open(output_path, 'w') as f:
                json.dump(config_data, f, indent=2)
            print(f"✅ JSON config template saved to: {output_path}")
            
        elif format_type.lower() in ["yaml", "yml"]:
            try:
                import yaml
                with open(output_path, 'w') as f:
                    yaml.dump(config_data, f, default_flow_style=False, indent=2)
                print(f"✅ YAML config template saved to: {output_path}")
            except ImportError:
                raise ImportError("PyYAML is required to create YAML config files. Install with: pip install pyyaml")
        else:
            raise ValueError(f"Unsupported format: {format_type}. Use 'json' or 'yaml'")
            
    except Exception as e:
        raise RuntimeError(f"Failed to create config template: {str(e)}")


def load_and_validate_config(config_path: Union[str, Path]) -> Dict[str, Any]:
    """
    Load and validate a configuration file.
    
    Args:
        config_path: Path to the configuration file
        
    Returns:
        Dictionary with validated configuration data
        
    Raises:
        FileNotFoundError: If config file doesn't exist
        ValueError: If config format is invalid
    """
    config_path = Path(config_path)
    
    if not config_path.exists():
        raise FileNotFoundError(f"Config file not found: {config_path}")
    
    print(f"📄 Loading and validating config: {config_path.name}")
    
    try:
        # Load based on extension
        if config_path.suffix.lower() == '.json':
            import json
            with open(config_path, 'r') as f:
                config_data = json.load(f)
        elif config_path.suffix.lower() in ['.yaml', '.yml']:
            try:
                import yaml
                with open(config_path, 'r') as f:
                    config_data = yaml.safe_load(f)
            except ImportError:
                raise ImportError("PyYAML required for YAML files. Install with: pip install pyyaml")
        else:
            raise ValueError(f"Unsupported config format: {config_path.suffix}")
        
        # Validate required structure
        metadata_section = config_data.get('metadata', {})
        valid_metadata_keys = {
            'image_threshold', 'pixel_threshold', 'pred_score_min',
            'pred_score_max', 'anomaly_map_min', 'anomaly_map_max'
        }
        
        # Check if at least one metadata parameter is provided
        provided_metadata = {k: v for k, v in metadata_section.items() 
                           if k in valid_metadata_keys and v is not None}
        
        if not provided_metadata:
            raise ValueError("Config must contain at least one valid metadata parameter")
        
        print(f"✅ Config validation passed: {len(provided_metadata)} metadata parameters found")
        
        return config_data
        
    except Exception as e:
        raise RuntimeError(f"Failed to load/validate config: {str(e)}")

In [ ]:
import os
from pathlib import Path
os.chdir('/home/ai_dsx.work/data/projects/be-vision-ad-tools/nbs')
Path.cwd()

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export('09_postprocessing.model_metadata.ipynb')